In [0]:
import sys
import importlib
sys.path.append('/Workspace/Users/luisfdiaz@bcp.com.pe/Modulo de Seguimiento/')

import SegScore as rmm
importlib.reload(rmm)

# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs,greatest
from pyspark.sql import functions as F
from pyspark import StorageLevel
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

In [0]:
def print_res(sparkf_df):
  import pandas as pd
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  pd_spark_df = sparkf_df.toPandas()
 
  return pd_spark_df

from collections import Counter

def find_repeated_columns(df):
    column_names = df.columns
    column_counts = Counter(column_names)
    repeated_columns = [col for col, count in column_counts.items() if count > 1]
    return repeated_columns

def find_missing_columns(df1, df2):
    columns_df1 = set(df1.columns)
    columns_df2 = set(df2.columns)
    
    missing_in_df2 = columns_df1 - columns_df2
    missing_in_df1 = columns_df2 - columns_df1
    
    print("Columnas en df1 pero no en df2:", missing_in_df2)
    print("Columnas en df2 pero no en df1:", missing_in_df1)

In [0]:
# hip = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_univ_pre_driver_app_hip')
# hip_pyme_0 = hip.filter("TIPO_FONDEO<>'FONDEO MVIV' AND CAMPANIA_AGRUPADA<> 'Ahorro Local' AND destipproductocreditohipotecario_trazer = 'TERCERA' AND CODMES < 202307")
# hip_pyme_1 = hip.filter("TIPO_FONDEO<>'FONDEO MVIV' AND CAMPANIA_AGRUPADA<> 'Ahorro Local' AND DES_PROD_HIP = 'Tercera' AND CODMES >= 202307")

# # Eliminamos la columna edad para no causar conflictos
# hip_pyme = hip_pyme_0.unionByName(hip_pyme_1) 
# hip_pyme = hip_pyme.drop("EDAD")

In [0]:
hip = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_univ_pre_driver_app_hip')

hip_pyme_0 = hip.filter(
    (col("TIPO_FONDEO") == 'FONDEO BCP') &
    (col("CAMPANIA_AGRUPADA") == 'Pyme') &
    (~col("DESTIPFINALIDADCREDHIPOTECARIO").isin(
        'AMPLIACION DE CREDITO PARA CONSTRUCCION',
        'AMPLIACION-REMOD.CASA CAMPO',
        'AMPLIACION-REMOD.CASA HABITACION',
        'Cambio de titularidad',
        'CAMBIO PLAZO',
        'CAMBIO VARIOS',
        'LIBRE DISP. CON GARANT.LOCAL COMERC.',
        'LIBRE DISPONIBILIDAD',
        'MIGRACION DE CREDITO',
        'MIGRACION DE MONEDA',
        'NUEVA AMPLIACION DE CREDITO',
        'Unificacion de credito hipotecario'
    )) & (col("CODMES")<202307)
)

hip_pyme_1 = hip.filter("TIPO_FONDEO<>'FONDEO MVIV' AND CAMPANIA_AGRUPADA<> 'Ahorro Local' AND DES_PROD_HIP = 'Tercera' AND CODMES >= 202307")

# Eliminamos la columna edad para no causar conflictos
hip_pyme = hip_pyme_0.unionByName(hip_pyme_1) 
hip_pyme = hip_pyme.drop("EDAD")

## Puntuaciones Tácticas

In [0]:
puntuaciones_total = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_hm_adm_hip_trad_score')
hip_pyme.createOrReplaceTempView('hip_pyme')
puntuaciones_total.createOrReplaceTempView('puntuaciones_total')
hip_pyme_1 = spark.sql("""
SELECT 
    A.*, 
    B.PD_APP_HIP_TRDPYME_2Q24,
    B.XB_APP_HIP_TRDPYME_2Q24,
    B.XB_APP_HIP_TRDPYME_IDIO,
    B.XB_ADC_POND85_F AS XB_BASE, 
    B.FLAG_PYME,
    B.EXP_PCT_EVOL_SHIP_U6M_RT_U24 AS DEU_NOHIP_6_24, 
    B.EXP_PCT_EVOL_SHIP_U6M_RT_U24_A3 AS DEU_NOHIP_6_24_A3,
    B.RCC_PCT_UTL6_UTL24_RT_U24 AS UTIL_PROM_6_24, 
    B.RCC_PCT_UTL6_UTL24_RT_U24_A2 AS UTIL_PROM_6_24_A2,
    B.SLD_PRM_PAS_MIN_24_24_RT_U24 AS PAS_MIN_PROM24, 
    B.SLD_PRM_PAS_MIN_24_24_RT_U24_I AS PAS_MIN_PROM24_I
FROM 
    hip_pyme AS A
LEFT JOIN 
    (SELECT distinct a.* FROM puntuaciones_total as a WHERE CAMPANIA_AGRUPADA_NEW = 'Pyme' and PD_APP_HIP_TRDPYME_2Q24 is not null) AS B 
ON 
    A.CODMES = B.CODMES AND A.CODCLAVECIC = B.CODCLAVECIC
""")

hip_pyme_1.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = hip_pyme_1.groupBy("codmes", "codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

## Base Espejo

In [0]:
espejo_total = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_mm_appscore_hip_trad')
hip_pyme_1.createOrReplaceTempView('hip_pyme_1')
espejo_total.createOrReplaceTempView('espejo_total')
hip_pyme_2 = spark.sql("""
SELECT A.*, 
       B.XB_ADC_POND85_F, 
       B.XB_ADC_POND85,
       B.XB_AD_HOC,
       B.XB_ACT_S_ADCW,
       B.XB_PAS_S_ADCW, 
       B.XB_TRX_S_ADCW,
       B.XB_RCC_S_ADCW,
       B.XB_DEM_S_ADCW,
       B.ANT_LABORAL_MESES,
       B.ANT_LABORAL_MESES_E2,
       B.CAT_ZONA1,
       B.CAT_ZONA1_C_ST,
       B.CIIU_DEF,
       B.CL_CIIU_ST,
       B.EDAD,
       B.EDAD_E1,
       B.EST_CIV,
       B.EST_CIV_ST,
       B.ING_CASCADA3,
       B.ING_CASCADA3_D2,
       B.XB_HIP,
       B.LTV_1,
       B.LTV_1_D2,
       B.GRUPO_PLAZO,
       B.GRUPO_PLAZO_ST,
       B.RIESGO_ZONA_ST,
       B.RIESGO_ZONA_ST_ST,
       B.XB_ADC_POND85_2,
       B.XB_AD_HOC_2,
       B.XB_DEM,
       B.XB_DEM_2,
       B.XB_DEM_S_ADCW_2,
       B.FLG_SOLTERO_C AS FLG_SOLTERO_ESPJ,
       B.SEGMENTO AS SEGMENTO_ESPJ,
       B.SEGMENTO2_st,
       B.FLG_NODEP AS FLG_NODEP_ESPJ,
       B.ZONA_PAUTA5,
       B.ZONA_PAUTA5_st,
       B.D_PYME,
       B.NUMHIJOS1_C2_ST,
       B.NUMHIJOS1,
       B.CL_VEH_MEAN_ST,
       B.ING_CASCADA2,
       CASE 
           WHEN B.ZONA_PAUTA5 IS NULL THEN '99. Missing'
           WHEN B.ZONA_PAUTA5 = 'LIMA TOP' THEN '1. Lima Top'
           WHEN B.ZONA_PAUTA5 = 'LIMA MODERNA' THEN '2. Lima Moderna'
           WHEN B.ZONA_PAUTA5 = 'PROVINCIA TOP' THEN '3. Provincia Top'
           WHEN B.ZONA_PAUTA5 = 'OTROS' THEN '4. Otros'
           ELSE '98. N.A.' 
       END AS ZONA_PAUTA5_F,
       B.PD_P7_ajus AS PD_BASE_ESPEJO,
       B.SC_P7_ajus AS SC_BASE_ESPEJO,
       B.XB_P7_ajus AS XB_BASE_ESPEJO
FROM hip_pyme_1 A
LEFT JOIN espejo_total B ON A.CODSOLICITUD = B.CODSOLICITUD
""")

hip_pyme_2.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = hip_pyme_2.groupBy("codmes", "codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
PD_SEG='PD_APP_HIP_TRDPYME_2Q24'
SCORE='SC_APP_PYME'
PAUTA=365

hip_pyme_2 = hip_pyme_2.withColumn("RANGO_SCORE", when(col(SCORE).isNull(), '99. Missing')
                   .when(col(SCORE) < 335, '01. <0 - 335>')
                   .when(col(SCORE) < 365, '02. [335 - 365>')
                   .when(col(SCORE) < 415, '03. [365 - 415>')
                   .when(col(SCORE) < 485, '04. [415 - 485>')
                   .when(col(SCORE) >= 485, '05. [485 - ++>')
                   .otherwise('98. Otro'))

# Add the RANGO_PD column
hip_pyme_2 = hip_pyme_2.withColumn("RANGO_PD", when(col(PD_SEG).isNull(), '99. Missing')
                   .when(col(PD_SEG) <= 0.0045641, '01. [0 - 0.45%]')
                   .when(col(PD_SEG) <= 0.0151879, '02. <0.45 - 1.51%]')
                   .when(col(PD_SEG) <= 0.0353823, '03. <1.51 - 3.53%]')
                   .when(col(PD_SEG) <= 0.0581041, '04. <3.53 - 5.81%]')
                   .when(col(PD_SEG) > 0.0581041, '05. <5.81 - 100%]')
                   .otherwise('98. Otro'))

# Add the MONTO_R column
hip_pyme_2 = hip_pyme_2.withColumn("MONTO_R", when(col("MTOAPROBADO_SOLES").isNull(), '99. Missing')
                   .when(col("MTOAPROBADO_SOLES") <= 237500, '1. [0 - 237,500]')
                   .when(col("MTOAPROBADO_SOLES") <= 340000, '2. <237,500 - 340,000]')
                   .when(col("MTOAPROBADO_SOLES") <= 460000, '3. <340,000 - 460,000]')
                   .when(col("MTOAPROBADO_SOLES") <= 680000, '4. <460,000 - 680,000]')
                   .otherwise('5. <680,000 - ++]'))

# Add the PAUTA column
hip_pyme_2 = hip_pyme_2.withColumn("PAUTA", when(col(SCORE) >= PAUTA, 1).otherwise(0))

# Add the MISS_PD column
hip_pyme_2 = hip_pyme_2.withColumn("MISS_PD", when(col(PD_SEG).isNull(), 1).otherwise(0))

# Add the MISS_MONTO column
hip_pyme_2 = hip_pyme_2.withColumn("MISS_MONTO", when(col("MTOAPROBADO_SOLES").isNull(), 1).otherwise(0))

# Add the FLG_ALTOS column
hip_pyme_2 = hip_pyme_2.withColumn("FLG_ALTOS", when(col("MTOAPROBADO_SOLES") >= 2166210, 1).otherwise(0))

# Add the DEU_NOHIP_6_24_F column
hip_pyme_2 = hip_pyme_2.withColumn("DEU_NOHIP_6_24_F", when(col("DEU_NOHIP_6_24").isin(
    None, 1111111111, -1111111111, 2222222222, -2222222222, 3333333333, -3333333333, 
    4444444444, 5555555555, 6666666666, 7777777777, 99999), None).otherwise(col("DEU_NOHIP_6_24")))

# Add the UTIL_PROM_6_24_F column
hip_pyme_2 = hip_pyme_2.withColumn("UTIL_PROM_6_24_F", when(col("UTIL_PROM_6_24").isin(
    None, 1111111111, -1111111111, 2222222222, -2222222222, 3333333333, -3333333333, 
    4444444444, 5555555555, 6666666666, 7777777777, 99999), None).otherwise(col("UTIL_PROM_6_24")))

# Add the PAS_MIN_PROM24_F column
hip_pyme_2 = hip_pyme_2.withColumn("PAS_MIN_PROM24_F", when(col("PAS_MIN_PROM24").isin(
    None, 1111111111, -1111111111, 2222222222, -2222222222, 3333333333, -3333333333, 
    4444444444, 5555555555, 6666666666, 7777777777, 99999), None).otherwise(col("PAS_MIN_PROM24")))

hip_pyme_2 = hip_pyme_2.withColumn(
    "INGRESO_CONY_TIT",
    coalesce(col("INGRESO_SOL_CONY"), expr("0")) + coalesce(col("INGRESO_RBM"), expr("0"))
)

hip_pyme_2 = hip_pyme_2.withColumn(
    "INGRESO_CONY_TIT",
    coalesce(col("INGRESO_SOL_CONY"), expr("0")) + coalesce(col("INGRESO_RBM"), expr("0"))
)

hip_pyme_2 = hip_pyme_2.withColumn(
    "CSOL_XB_RBM", (174.25 - F.col("CSOL_SC_RBM")) / 57.708
)

hip_pyme_2 = hip_pyme_2.withColumn("CSOL_PD_RBM", 1 / (1 + F.exp(-F.col("CSOL_XB_RBM"))))

hip_pyme_2 = hip_pyme_2.withColumn("PD_RBM", coalesce(col("PD_ESTADOS"), col("CSOL_PD_RBM")))


ltv_malita = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_EXT_SEG_AGRUP_LTV_202504')

hip_pyme_2 = hip_pyme_2.alias("A") \
    .join(ltv_malita.alias("F"), F.col("A.CODSOLICITUD") == F.col("F.CODSOLICITUD"), "left") \
    .join(hip_pyme_2.alias("G"), (F.col("A.CODSOLICITUD") == F.col("G.CODSOLICITUD")) & (F.col("A.CODCLAVEOPECTA") == F.col("G.CODCLAVEOPECTA")), "left") \
    .select("A.*", F.coalesce(F.col("F.LTV"), F.col("G.LTV_RBM")).alias("LTV_CORREGIDO_0") ,F.col("F.LTV").alias("LTV_EXT"))

hip_pyme_2 = hip_pyme_2.withColumn("LTV_CORREGIDO_VF", when(col("CODMES") >= 202308, col("NFT_LTV")).otherwise(col("LTV_CORREGIDO_0")))

hip_pyme_2 = hip_pyme_2.withColumn(
    "RAN_PLAZO",
    F.when(F.col("RAN_PLAZO") == "HASTA 10", "1  HASTA 10")
     .when(F.col("RAN_PLAZO") == "<10 - 15]", "2 <10 - 15]")
     .when(F.col("RAN_PLAZO") == "<15 - 20]", "3 <15 - 20]")
     .when(F.col("RAN_PLAZO") == "MAS DE 20", "4 MAS DE 20")
     .otherwise(F.col("RAN_PLAZO"))
)

hip_pyme_2 = hip_pyme_2.withColumn("SUITE_CAT_ZONA", 
                   when(col("SUITE_desprovincia") == 'LIMA', 'Lima')
                   .when(col("SUITE_desprovincia").isNull(), '99. Missing')
                   .otherwise('Provincia'))

hip_pyme_2 = hip_pyme_2.withColumn(
    "SUITE_Estcivil_J", 
    when(col("codmes") <= 202307, col("EST_CIV")).otherwise(col("SUITE_Estcivil"))
)

### Pegando actividad economica

In [0]:
from pyspark.sql.functions import date_format, concat, lit, lpad

md_cliente = spark.table('catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_fuenteinfocli_vu.md_clienteriesgos')
hip_pyme_2 = hip_pyme_2.join(
    md_cliente.selectExpr("codclavepartycli as codclavepartycliempleador", "tipactividadeconomica", "destipactividadeconomica"), 
    on = "codclavepartycliempleador",
    how = "left"
)

hip_pyme_2.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = hip_pyme_2.groupBy("codmes", "codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
hip_pyme_2 = hip_pyme_2.withColumn("NFT_GRUPO_CIIU", 
when(col('TIPACTIVIDADECONOMICA') =='0111' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0112' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0113' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0114' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0115' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0119' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0121' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0122' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0123' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0124' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0125' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0126' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0127' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0128' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0129' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0130' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0141' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0142' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0143' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0144' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0145' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0149' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0150' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0161' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0162' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0163' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0164' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0170' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0210' , '02')
.when(col('TIPACTIVIDADECONOMICA') =='0220' , '02')
.when(col('TIPACTIVIDADECONOMICA') =='0230' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0240' , '02')
.when(col('TIPACTIVIDADECONOMICA') =='0311' , '05')
.when(col('TIPACTIVIDADECONOMICA') =='0312' , '05')
.when(col('TIPACTIVIDADECONOMICA') =='0321' , '05')
.when(col('TIPACTIVIDADECONOMICA') =='0322' , '05')
.when(col('TIPACTIVIDADECONOMICA') =='0510' , '10')
.when(col('TIPACTIVIDADECONOMICA') =='0520' , '10')
.when(col('TIPACTIVIDADECONOMICA') =='0610' , '11')
.when(col('TIPACTIVIDADECONOMICA') =='0620' , '11')
.when(col('TIPACTIVIDADECONOMICA') =='0710' , '13')
.when(col('TIPACTIVIDADECONOMICA') =='0721' , '12')
.when(col('TIPACTIVIDADECONOMICA') =='0722' , '13')
.when(col('TIPACTIVIDADECONOMICA') =='0723' , '13')
.when(col('TIPACTIVIDADECONOMICA') =='0729' , '13')
.when(col('TIPACTIVIDADECONOMICA') =='0811' , '14')
.when(col('TIPACTIVIDADECONOMICA') =='0812' , '14')
.when(col('TIPACTIVIDADECONOMICA') =='0820' , '14')
.when(col('TIPACTIVIDADECONOMICA') =='0891' , '14')
.when(col('TIPACTIVIDADECONOMICA') =='0892' , '14')
.when(col('TIPACTIVIDADECONOMICA') =='0899' , '10')
.when(col('TIPACTIVIDADECONOMICA') =='0910' , '11')
.when(col('TIPACTIVIDADECONOMICA') =='0990' , '10')
.when(col('TIPACTIVIDADECONOMICA') =='1011' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1012' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1020' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1030' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1040' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1051' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1052' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1061' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1062' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1063' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1071' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1072' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1081' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1082' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1083' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1084' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1089' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1090' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1101' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1102' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1103' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1104' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1200' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='1311' , '17')
.when(col('TIPACTIVIDADECONOMICA') =='1312' , '17')
.when(col('TIPACTIVIDADECONOMICA') =='1313' , '17')
.when(col('TIPACTIVIDADECONOMICA') =='1313' , '18')
.when(col('TIPACTIVIDADECONOMICA') =='1391' , '17')
.when(col('TIPACTIVIDADECONOMICA') =='1392' , '17')
.when(col('TIPACTIVIDADECONOMICA') =='1393' , '17')
.when(col('TIPACTIVIDADECONOMICA') =='1394' , '17')
.when(col('TIPACTIVIDADECONOMICA') =='1399' , '17')
.when(col('TIPACTIVIDADECONOMICA') =='1410' , '18')
.when(col('TIPACTIVIDADECONOMICA') =='1420' , '18')
.when(col('TIPACTIVIDADECONOMICA') =='1430' , '17')
.when(col('TIPACTIVIDADECONOMICA') =='1511' , '18')
.when(col('TIPACTIVIDADECONOMICA') =='1512' , '19')
.when(col('TIPACTIVIDADECONOMICA') =='1513' , '19')
.when(col('TIPACTIVIDADECONOMICA') =='1521' , '19')
.when(col('TIPACTIVIDADECONOMICA') =='1522' , '19')
.when(col('TIPACTIVIDADECONOMICA') =='1523' , '19')
.when(col('TIPACTIVIDADECONOMICA') =='1610' , '20')
.when(col('TIPACTIVIDADECONOMICA') =='1620' , '20')
.when(col('TIPACTIVIDADECONOMICA') =='1630' , '20')
.when(col('TIPACTIVIDADECONOMICA') =='1640' , '20')
.when(col('TIPACTIVIDADECONOMICA') =='1690' , '19')
.when(col('TIPACTIVIDADECONOMICA') =='1701' , '21')
.when(col('TIPACTIVIDADECONOMICA') =='1702' , '21')
.when(col('TIPACTIVIDADECONOMICA') =='1709' , '17')
.when(col('TIPACTIVIDADECONOMICA') =='1811' , '17')
.when(col('TIPACTIVIDADECONOMICA') =='1812' , '22')
.when(col('TIPACTIVIDADECONOMICA') =='1820' , '22')
.when(col('TIPACTIVIDADECONOMICA') =='1910' , '23')
.when(col('TIPACTIVIDADECONOMICA') =='1921' , '10')
.when(col('TIPACTIVIDADECONOMICA') =='2011' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='2012' , '24')
.when(col('TIPACTIVIDADECONOMICA') =='2013' , '24')
.when(col('TIPACTIVIDADECONOMICA') =='2014' , '24')
.when(col('TIPACTIVIDADECONOMICA') =='2021' , '24')
.when(col('TIPACTIVIDADECONOMICA') =='2022' , '24')
.when(col('TIPACTIVIDADECONOMICA') =='2023' , '24')
.when(col('TIPACTIVIDADECONOMICA') =='2029' , '24')
.when(col('TIPACTIVIDADECONOMICA') =='2030' , '24')
.when(col('TIPACTIVIDADECONOMICA') =='2100' , '23')
.when(col('TIPACTIVIDADECONOMICA') =='2211' , '25')
.when(col('TIPACTIVIDADECONOMICA') =='2212' , '25')
.when(col('TIPACTIVIDADECONOMICA') =='2219' , '19')
.when(col('TIPACTIVIDADECONOMICA') =='2221' , '25')
.when(col('TIPACTIVIDADECONOMICA') =='2229' , '19')
.when(col('TIPACTIVIDADECONOMICA') =='2310' , '26')
.when(col('TIPACTIVIDADECONOMICA') =='2391' , '26')
.when(col('TIPACTIVIDADECONOMICA') =='2392' , '26')
.when(col('TIPACTIVIDADECONOMICA') =='2393' , '26')
.when(col('TIPACTIVIDADECONOMICA') =='2394' , '26')
.when(col('TIPACTIVIDADECONOMICA') =='2395' , '26')
.when(col('TIPACTIVIDADECONOMICA') =='2396' , '26')
.when(col('TIPACTIVIDADECONOMICA') =='2399' , '26')
.when(col('TIPACTIVIDADECONOMICA') =='2410' , '27')
.when(col('TIPACTIVIDADECONOMICA') =='2421' , '27')
.when(col('TIPACTIVIDADECONOMICA') =='2429' , '23')
.when(col('TIPACTIVIDADECONOMICA') =='2431' , '27')
.when(col('TIPACTIVIDADECONOMICA') =='2432' , '27')
.when(col('TIPACTIVIDADECONOMICA') =='2511' , '28')
.when(col('TIPACTIVIDADECONOMICA') =='2512' , '28')
.when(col('TIPACTIVIDADECONOMICA') =='2513' , '28')
.when(col('TIPACTIVIDADECONOMICA') =='2520' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='2591' , '28')
.when(col('TIPACTIVIDADECONOMICA') =='2592' , '28')
.when(col('TIPACTIVIDADECONOMICA') =='2593' , '28')
.when(col('TIPACTIVIDADECONOMICA') =='2599' , '28')
.when(col('TIPACTIVIDADECONOMICA') =='2610' , '25')
.when(col('TIPACTIVIDADECONOMICA') =='2620' , '30')
.when(col('TIPACTIVIDADECONOMICA') =='2630' , '31')
.when(col('TIPACTIVIDADECONOMICA') =='2640' , '32')
.when(col('TIPACTIVIDADECONOMICA') =='2651' , '31')
.when(col('TIPACTIVIDADECONOMICA') =='2652' , '33')
.when(col('TIPACTIVIDADECONOMICA') =='2660' , '33')
.when(col('TIPACTIVIDADECONOMICA') =='2670' , '32')
.when(col('TIPACTIVIDADECONOMICA') =='2680' , '24')
.when(col('TIPACTIVIDADECONOMICA') =='2711' , '31')
.when(col('TIPACTIVIDADECONOMICA') =='2712' , '31')
.when(col('TIPACTIVIDADECONOMICA') =='2720' , '31')
.when(col('TIPACTIVIDADECONOMICA') =='2731' , '31')
.when(col('TIPACTIVIDADECONOMICA') =='2732' , '25')
.when(col('TIPACTIVIDADECONOMICA') =='2740' , '31')
.when(col('TIPACTIVIDADECONOMICA') =='2750' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='2790' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='2811' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='2812' , '25')
.when(col('TIPACTIVIDADECONOMICA') =='2813' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='2814' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='2815' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='2816' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='2817' , '24')
.when(col('TIPACTIVIDADECONOMICA') =='2818' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='2819' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='2821' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='2822' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='2823' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='2824' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='2825' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='2826' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='2829' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='2910' , '34')
.when(col('TIPACTIVIDADECONOMICA') =='2920' , '34')
.when(col('TIPACTIVIDADECONOMICA') =='2930' , '31')
.when(col('TIPACTIVIDADECONOMICA') =='3011' , '35')
.when(col('TIPACTIVIDADECONOMICA') =='3012' , '35')
.when(col('TIPACTIVIDADECONOMICA') =='3020' , '35')
.when(col('TIPACTIVIDADECONOMICA') =='3030' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='3040' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='3091' , '35')
.when(col('TIPACTIVIDADECONOMICA') =='3092' , '35')
.when(col('TIPACTIVIDADECONOMICA') =='3099' , '35')
.when(col('TIPACTIVIDADECONOMICA') =='3110' , '35')
.when(col('TIPACTIVIDADECONOMICA') =='3120' , '36')
.when(col('TIPACTIVIDADECONOMICA') =='3210' , '33')
.when(col('TIPACTIVIDADECONOMICA') =='3220' , '36')
.when(col('TIPACTIVIDADECONOMICA') =='3230' , '19')
.when(col('TIPACTIVIDADECONOMICA') =='3240' , '35')
.when(col('TIPACTIVIDADECONOMICA') =='3250' , '17')
.when(col('TIPACTIVIDADECONOMICA') =='3290' , '18')
.when(col('TIPACTIVIDADECONOMICA') =='3311' , '28')
.when(col('TIPACTIVIDADECONOMICA') =='3312' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='3313' , '31')
.when(col('TIPACTIVIDADECONOMICA') =='3314' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='3315' , '35')
.when(col('TIPACTIVIDADECONOMICA') =='3319' , '17')
.when(col('TIPACTIVIDADECONOMICA') =='3320' , '29')
.when(col('TIPACTIVIDADECONOMICA') =='3511' , '40')
.when(col('TIPACTIVIDADECONOMICA') =='3512' , '40')
.when(col('TIPACTIVIDADECONOMICA') =='3513' , '40')
.when(col('TIPACTIVIDADECONOMICA') =='3514' , '40')
.when(col('TIPACTIVIDADECONOMICA') =='3520' , '40')
.when(col('TIPACTIVIDADECONOMICA') =='3530' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='3600' , '41')
.when(col('TIPACTIVIDADECONOMICA') =='3700' , '90')
.when(col('TIPACTIVIDADECONOMICA') =='3811' , '90')
.when(col('TIPACTIVIDADECONOMICA') =='3812' , '23')
.when(col('TIPACTIVIDADECONOMICA') =='3821' , '24')
.when(col('TIPACTIVIDADECONOMICA') =='3822' , '23')
.when(col('TIPACTIVIDADECONOMICA') =='3830' , '37')
.when(col('TIPACTIVIDADECONOMICA') =='3900' , '45')
.when(col('TIPACTIVIDADECONOMICA') =='4111' , '45')
.when(col('TIPACTIVIDADECONOMICA') =='4112' , '45')
.when(col('TIPACTIVIDADECONOMICA') =='4210' , '45')
.when(col('TIPACTIVIDADECONOMICA') =='4220' , '45')
.when(col('TIPACTIVIDADECONOMICA') =='4290' , '45')
.when(col('TIPACTIVIDADECONOMICA') =='4311' , '45')
.when(col('TIPACTIVIDADECONOMICA') =='4312' , '45')
.when(col('TIPACTIVIDADECONOMICA') =='4321' , '45')
.when(col('TIPACTIVIDADECONOMICA') =='4322' , '45')
.when(col('TIPACTIVIDADECONOMICA') =='4329' , '45')
.when(col('TIPACTIVIDADECONOMICA') =='4330' , '45')
.when(col('TIPACTIVIDADECONOMICA') =='4390' , '45')
.when(col('TIPACTIVIDADECONOMICA') =='4511' , '50')
.when(col('TIPACTIVIDADECONOMICA') =='4512' , '50')
.when(col('TIPACTIVIDADECONOMICA') =='4520' , '50')
.when(col('TIPACTIVIDADECONOMICA') =='4530' , '50')
.when(col('TIPACTIVIDADECONOMICA') =='4541' , '50')
.when(col('TIPACTIVIDADECONOMICA') =='4542' , '35')
.when(col('TIPACTIVIDADECONOMICA') =='4610' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4620' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4631' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4632' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4641' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4642' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4643' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4644' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4645' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4649' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4651' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4652' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4653' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4659' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4661' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4662' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4663' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4664' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4665' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4669' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4690' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4711' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4719' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4721' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4722' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4723' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4724' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4729' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4731' , '50')
.when(col('TIPACTIVIDADECONOMICA') =='4732' , '50')
.when(col('TIPACTIVIDADECONOMICA') =='4741' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4742' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4751' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4752' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4753' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4754' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4755' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4759' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4761' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4762' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4769' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4771' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4772' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4773' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4774' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4775' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4781' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4782' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4789' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4791' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4792' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4799' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4911' , '60')
.when(col('TIPACTIVIDADECONOMICA') =='4912' , '60')
.when(col('TIPACTIVIDADECONOMICA') =='4921' , '60')
.when(col('TIPACTIVIDADECONOMICA') =='4922' , '60')
.when(col('TIPACTIVIDADECONOMICA') =='4923' , '60')
.when(col('TIPACTIVIDADECONOMICA') =='4930' , '60')
.when(col('TIPACTIVIDADECONOMICA') =='5011' , '61')
.when(col('TIPACTIVIDADECONOMICA') =='5012' , '61')
.when(col('TIPACTIVIDADECONOMICA') =='5021' , '61')
.when(col('TIPACTIVIDADECONOMICA') =='5022' , '61')
.when(col('TIPACTIVIDADECONOMICA') =='5111' , '62')
.when(col('TIPACTIVIDADECONOMICA') =='5112' , '62')
.when(col('TIPACTIVIDADECONOMICA') =='5121' , '62')
.when(col('TIPACTIVIDADECONOMICA') =='5122' , '62')
.when(col('TIPACTIVIDADECONOMICA') =='5210' , '63')
.when(col('TIPACTIVIDADECONOMICA') =='5221' , '50')
.when(col('TIPACTIVIDADECONOMICA') =='5222' , '63')
.when(col('TIPACTIVIDADECONOMICA') =='5223' , '63')
.when(col('TIPACTIVIDADECONOMICA') =='5224' , '63')
.when(col('TIPACTIVIDADECONOMICA') =='5229' , '63')
.when(col('TIPACTIVIDADECONOMICA') =='5310' , '64')
.when(col('TIPACTIVIDADECONOMICA') =='5320' , '64')
.when(col('TIPACTIVIDADECONOMICA') =='5511' , '55')
.when(col('TIPACTIVIDADECONOMICA') =='5512' , '55')
.when(col('TIPACTIVIDADECONOMICA') =='5513' , '55')
.when(col('TIPACTIVIDADECONOMICA') =='5514' , '55')
.when(col('TIPACTIVIDADECONOMICA') =='5519' , '55')
.when(col('TIPACTIVIDADECONOMICA') =='5520' , '55')
.when(col('TIPACTIVIDADECONOMICA') =='5530' , '55')
.when(col('TIPACTIVIDADECONOMICA') =='5590' , '55')
.when(col('TIPACTIVIDADECONOMICA') =='5611' , '55')
.when(col('TIPACTIVIDADECONOMICA') =='5612' , '55')
.when(col('TIPACTIVIDADECONOMICA') =='5613' , '55')
.when(col('TIPACTIVIDADECONOMICA') =='5619' , '55')
.when(col('TIPACTIVIDADECONOMICA') =='5621' , '55')
.when(col('TIPACTIVIDADECONOMICA') =='5629' , '55')
.when(col('TIPACTIVIDADECONOMICA') =='5630' , '55')
.when(col('TIPACTIVIDADECONOMICA') =='5811' , '22')
.when(col('TIPACTIVIDADECONOMICA') =='5812' , '22')
.when(col('TIPACTIVIDADECONOMICA') =='5813' , '22')
.when(col('TIPACTIVIDADECONOMICA') =='5819' , '22')
.when(col('TIPACTIVIDADECONOMICA') =='5820' , '72')
.when(col('TIPACTIVIDADECONOMICA') =='5911' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='5912' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='5913' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='5914' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='5920' , '22')
.when(col('TIPACTIVIDADECONOMICA') =='6010' , '72')
.when(col('TIPACTIVIDADECONOMICA') =='6020' , '72')
.when(col('TIPACTIVIDADECONOMICA') =='6110' , '64')
.when(col('TIPACTIVIDADECONOMICA') =='6120' , '64')
.when(col('TIPACTIVIDADECONOMICA') =='6130' , '64')
.when(col('TIPACTIVIDADECONOMICA') =='6190' , '64')
.when(col('TIPACTIVIDADECONOMICA') =='6201' , '72')
.when(col('TIPACTIVIDADECONOMICA') =='6202' , '72')
.when(col('TIPACTIVIDADECONOMICA') =='6209' , '72')
.when(col('TIPACTIVIDADECONOMICA') =='6311' , '72')
.when(col('TIPACTIVIDADECONOMICA') =='6312' , '72')
.when(col('TIPACTIVIDADECONOMICA') =='6391' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='6399' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='6411' , '65')
.when(col('TIPACTIVIDADECONOMICA') =='6412' , '65')
.when(col('TIPACTIVIDADECONOMICA') =='6421' , '65')
.when(col('TIPACTIVIDADECONOMICA') =='6422' , '65')
.when(col('TIPACTIVIDADECONOMICA') =='6423' , '65')
.when(col('TIPACTIVIDADECONOMICA') =='6424' , '65')
.when(col('TIPACTIVIDADECONOMICA') =='6431' , '67')
.when(col('TIPACTIVIDADECONOMICA') =='6432' , '66')
.when(col('TIPACTIVIDADECONOMICA') =='6491' , '65')
.when(col('TIPACTIVIDADECONOMICA') =='6492' , '65')
.when(col('TIPACTIVIDADECONOMICA') =='6493' , '65')
.when(col('TIPACTIVIDADECONOMICA') =='6494' , '65')
.when(col('TIPACTIVIDADECONOMICA') =='6495' , '65')
.when(col('TIPACTIVIDADECONOMICA') =='6499' , '65')
.when(col('TIPACTIVIDADECONOMICA') =='6511' , '66')
.when(col('TIPACTIVIDADECONOMICA') =='6512' , '66')
.when(col('TIPACTIVIDADECONOMICA') =='6513' , '66')
.when(col('TIPACTIVIDADECONOMICA') =='6514' , '65')
.when(col('TIPACTIVIDADECONOMICA') =='6521' , '66')
.when(col('TIPACTIVIDADECONOMICA') =='6522' , '66')
.when(col('TIPACTIVIDADECONOMICA') =='6531' , '66')
.when(col('TIPACTIVIDADECONOMICA') =='6532' , '66')
.when(col('TIPACTIVIDADECONOMICA') =='6611' , '67')
.when(col('TIPACTIVIDADECONOMICA') =='6612' , '67')
.when(col('TIPACTIVIDADECONOMICA') =='6613' , '67')
.when(col('TIPACTIVIDADECONOMICA') =='6614' , '67')
.when(col('TIPACTIVIDADECONOMICA') =='6615' , '67')
.when(col('TIPACTIVIDADECONOMICA') =='6619' , '67')
.when(col('TIPACTIVIDADECONOMICA') =='6621' , '67')
.when(col('TIPACTIVIDADECONOMICA') =='6629' , '67')
.when(col('TIPACTIVIDADECONOMICA') =='6630' , '67')
.when(col('TIPACTIVIDADECONOMICA') =='6810' , '70')
.when(col('TIPACTIVIDADECONOMICA') =='6820' , '70')
.when(col('TIPACTIVIDADECONOMICA') =='6910' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='6920' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='7010' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='7020' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='7110' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='7120' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='7210' , '73')
.when(col('TIPACTIVIDADECONOMICA') =='7220' , '73')
.when(col('TIPACTIVIDADECONOMICA') =='7310' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='7320' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='7410' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='7420' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='7490' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='7500' , '85')
.when(col('TIPACTIVIDADECONOMICA') =='7710' , '71')
.when(col('TIPACTIVIDADECONOMICA') =='7721' , '71')
.when(col('TIPACTIVIDADECONOMICA') =='7722' , '71')
.when(col('TIPACTIVIDADECONOMICA') =='7729' , '71')
.when(col('TIPACTIVIDADECONOMICA') =='7730' , '71')
.when(col('TIPACTIVIDADECONOMICA') =='7740' , '65')
.when(col('TIPACTIVIDADECONOMICA') =='7810' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='7820' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='7830' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='7911' , '63')
.when(col('TIPACTIVIDADECONOMICA') =='7912' , '63')
.when(col('TIPACTIVIDADECONOMICA') =='7990' , '63')
.when(col('TIPACTIVIDADECONOMICA') =='8010' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='8020' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='8030' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='8110' , '70')
.when(col('TIPACTIVIDADECONOMICA') =='8121' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='8129' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='8130' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='8211' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='8219' , '64')
.when(col('TIPACTIVIDADECONOMICA') =='8220' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='8230' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='8291' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='8292' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='8299' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='8411' , '75')
.when(col('TIPACTIVIDADECONOMICA') =='8412' , '75')
.when(col('TIPACTIVIDADECONOMICA') =='8413' , '75')
.when(col('TIPACTIVIDADECONOMICA') =='8414' , '75')
.when(col('TIPACTIVIDADECONOMICA') =='8415' , '75')
.when(col('TIPACTIVIDADECONOMICA') =='8421' , '75')
.when(col('TIPACTIVIDADECONOMICA') =='8422' , '75')
.when(col('TIPACTIVIDADECONOMICA') =='8423' , '75')
.when(col('TIPACTIVIDADECONOMICA') =='8424' , '75')
.when(col('TIPACTIVIDADECONOMICA') =='8430' , '75')
.when(col('TIPACTIVIDADECONOMICA') =='8512' , '80')
.when(col('TIPACTIVIDADECONOMICA') =='8513' , '80')
.when(col('TIPACTIVIDADECONOMICA') =='8521' , '80')
.when(col('TIPACTIVIDADECONOMICA') =='8522' , '80')
.when(col('TIPACTIVIDADECONOMICA') =='8523' , '80')
.when(col('TIPACTIVIDADECONOMICA') =='8530' , '80')
.when(col('TIPACTIVIDADECONOMICA') =='8541' , '80')
.when(col('TIPACTIVIDADECONOMICA') =='8542' , '80')
.when(col('TIPACTIVIDADECONOMICA') =='8543' , '80')
.when(col('TIPACTIVIDADECONOMICA') =='8544' , '80')
.when(col('TIPACTIVIDADECONOMICA') =='8551' , '80')
.when(col('TIPACTIVIDADECONOMICA') =='8552' , '80')
.when(col('TIPACTIVIDADECONOMICA') =='8553' , '80')
.when(col('TIPACTIVIDADECONOMICA') =='8559' , '80')
.when(col('TIPACTIVIDADECONOMICA') =='8560' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='8610' , '85')
.when(col('TIPACTIVIDADECONOMICA') =='8621' , '85')
.when(col('TIPACTIVIDADECONOMICA') =='8622' , '85')
.when(col('TIPACTIVIDADECONOMICA') =='8691' , '85')
.when(col('TIPACTIVIDADECONOMICA') =='8692' , '85')
.when(col('TIPACTIVIDADECONOMICA') =='8699' , '85')
.when(col('TIPACTIVIDADECONOMICA') =='8710' , '85')
.when(col('TIPACTIVIDADECONOMICA') =='8720' , '85')
.when(col('TIPACTIVIDADECONOMICA') =='8730' , '85')
.when(col('TIPACTIVIDADECONOMICA') =='8790' , '85')
.when(col('TIPACTIVIDADECONOMICA') =='8810' , '85')
.when(col('TIPACTIVIDADECONOMICA') =='8890' , '85')
.when(col('TIPACTIVIDADECONOMICA') =='9001' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='9002' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='9003' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='9004' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='9005' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='9006' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='9007' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='9008' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='9101' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='9102' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='9103' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='9200' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='9311' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='9312' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='9319' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='9321' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='9329' , '92')
.when(col('TIPACTIVIDADECONOMICA') =='9411' , '91')
.when(col('TIPACTIVIDADECONOMICA') =='9412' , '91')
.when(col('TIPACTIVIDADECONOMICA') =='9420' , '91')
.when(col('TIPACTIVIDADECONOMICA') =='9491' , '91')
.when(col('TIPACTIVIDADECONOMICA') =='9492' , '91')
.when(col('TIPACTIVIDADECONOMICA') =='9499' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='9511' , '72')
.when(col('TIPACTIVIDADECONOMICA') =='9512' , '32')
.when(col('TIPACTIVIDADECONOMICA') =='9521' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='9522' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='9523' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='9524' , '36')
.when(col('TIPACTIVIDADECONOMICA') =='9529' , '36')
.when(col('TIPACTIVIDADECONOMICA') =='9601' , '93')
.when(col('TIPACTIVIDADECONOMICA') =='9602' , '93')
.when(col('TIPACTIVIDADECONOMICA') =='9603' , '93')
.when(col('TIPACTIVIDADECONOMICA') =='9609' , '93')
.when(col('TIPACTIVIDADECONOMICA') =='9700' , '95')
.when(col('TIPACTIVIDADECONOMICA') =='9810' , '96')
.when(col('TIPACTIVIDADECONOMICA') =='9820' , '97')
.when(col('TIPACTIVIDADECONOMICA') =='9900' , '99')
.when(col('TIPACTIVIDADECONOMICA') =='0116' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0146' , '01')
.when(col('TIPACTIVIDADECONOMICA') =='0810' , '14')
.when(col('TIPACTIVIDADECONOMICA') =='0893' , '14')
.when(col('TIPACTIVIDADECONOMICA') =='1010' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1050' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1073' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1074' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1075' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1079' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1080' , '15')
.when(col('TIPACTIVIDADECONOMICA') =='1520' , '19')
.when(col('TIPACTIVIDADECONOMICA') =='1621' , '20')
.when(col('TIPACTIVIDADECONOMICA') =='1622' , '20')
.when(col('TIPACTIVIDADECONOMICA') =='1623' , '20')
.when(col('TIPACTIVIDADECONOMICA') =='1629' , '20')
.when(col('TIPACTIVIDADECONOMICA') =='1920' , '23')
.when(col('TIPACTIVIDADECONOMICA') =='2220' , '25')
.when(col('TIPACTIVIDADECONOMICA') =='2420' , '27')
.when(col('TIPACTIVIDADECONOMICA') =='2710' , '31')
.when(col('TIPACTIVIDADECONOMICA') =='2733' , '31')
.when(col('TIPACTIVIDADECONOMICA') =='3100' , '36')
.when(col('TIPACTIVIDADECONOMICA') =='3211' , '36')
.when(col('TIPACTIVIDADECONOMICA') =='3212' , '36')
.when(col('TIPACTIVIDADECONOMICA') =='3510' , '40')
.when(col('TIPACTIVIDADECONOMICA') =='4100' , '45')
.when(col('TIPACTIVIDADECONOMICA') =='4510' , '50')
.when(col('TIPACTIVIDADECONOMICA') =='4540' , '50')
.when(col('TIPACTIVIDADECONOMICA') =='4630' , '51')
.when(col('TIPACTIVIDADECONOMICA') =='4730' , '50')
.when(col('TIPACTIVIDADECONOMICA') =='4763' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='4764' , '52')
.when(col('TIPACTIVIDADECONOMICA') =='5110' , '62')
.when(col('TIPACTIVIDADECONOMICA') =='5120' , '62')
.when(col('TIPACTIVIDADECONOMICA') =='5510' , '55')
.when(col('TIPACTIVIDADECONOMICA') =='5610' , '55')
.when(col('TIPACTIVIDADECONOMICA') =='6419' , '65')
.when(col('TIPACTIVIDADECONOMICA') =='6420' , '65')
.when(col('TIPACTIVIDADECONOMICA') =='6430' , '65')
.when(col('TIPACTIVIDADECONOMICA') =='6520' , '66')
.when(col('TIPACTIVIDADECONOMICA') =='6530' , '66')
.when(col('TIPACTIVIDADECONOMICA') =='6622' , '67')
.when(col('TIPACTIVIDADECONOMICA') =='8510' , '80')
.when(col('TIPACTIVIDADECONOMICA') =='8549' , '80')
.when(col('TIPACTIVIDADECONOMICA') =='8550' , '74')
.when(col('TIPACTIVIDADECONOMICA') =='8620' , '85')
.when(col('TIPACTIVIDADECONOMICA') =='8690' , '85')
)


hip_pyme_2 = hip_pyme_2.withColumn("NFT_CIIU_DEF", 
    when(col("NFT_GRUPO_CIIU").isin("01", "02"), 1)
    .when(col("NFT_GRUPO_CIIU") == "05", 2)
    .when(col("NFT_GRUPO_CIIU").isin("10", "11", "12", "13", "14"), 3)
    .when(col("NFT_GRUPO_CIIU").isin(
        "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28",
        "29", "30", "31", "32", "33", "34", "35", "36", "37"), 4)
    .when(col("NFT_GRUPO_CIIU").isin("40", "41"), 5)
    .when(col("NFT_GRUPO_CIIU") == "45", 6)
    .when(col("NFT_GRUPO_CIIU").isin("50", "51", "52"), 7)
    .when(col("NFT_GRUPO_CIIU") == "55", 8)
    .when(col("NFT_GRUPO_CIIU").isin("60", "61", "62", "63", "64"), 9)
    .when(col("NFT_GRUPO_CIIU").isin("65", "66", "67"), 10)
    .when(col("NFT_GRUPO_CIIU") == "70", 11)
    .when(col("NFT_GRUPO_CIIU").isin("71", "72", "73", "75", "80", "85", "90", "91", "92", "93", "95", "99"), 12)
    .when(col("NFT_GRUPO_CIIU") == "74", 13)
)

hip_pyme_2 = hip_pyme_2.withColumn("NFT_ID_CIIU",
    when(col("NFT_CIIU_DEF") == 1, "AGROPECUARIO")
    .when(col("NFT_CIIU_DEF") == 2, "PESCA")
    .when(col("NFT_CIIU_DEF") == 3, "MINERIA E HIDROCARBUROS")
    .when(col("NFT_CIIU_DEF") == 4, "MANUFACTURA")
    .when(col("NFT_CIIU_DEF") == 5, "SUMINISTRO DE SERV PUBLICOS")
    .when(col("NFT_CIIU_DEF") == 6, "CONSTRUCCION")
    .when(col("NFT_CIIU_DEF") == 7, "COMERCIO")
    .when(col("NFT_CIIU_DEF") == 8, "HOTELES Y RESTAURANTES")
    .when(col("NFT_CIIU_DEF") == 9, "TRANSPORTE Y COMUNICACIONES")
    .when(col("NFT_CIIU_DEF") == 10, "INTERMEDIACION FINANCIERA")
    .when(col("NFT_CIIU_DEF") == 11, "INMOBILIARIAS")
    .when(col("NFT_CIIU_DEF") == 12, "OTROS SERVICIOS")
    .when(col("NFT_CIIU_DEF") == 13, "OTRAS ACTIVIDADES EMPRESARIALES")
)

In [0]:
hip_pyme_2 = hip_pyme_2.join(
    md_cliente.selectExpr("codinternocomputacional", "tipactividadeconomica as tipactividadeconomica_0", "destipactividadeconomica as destipactividadeconomica_0"), 
    on='codinternocomputacional',
    how='left'
)

hip_pyme_2.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = hip_pyme_2.groupBy("codmes", "codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
hip_pyme_2 = hip_pyme_2.withColumn("ANT_GRUPO_CIIU", 
when(col('tipactividadeconomica_0') =='0111' , '01')
.when(col('tipactividadeconomica_0') =='0112' , '01')
.when(col('tipactividadeconomica_0') =='0113' , '01')
.when(col('tipactividadeconomica_0') =='0114' , '01')
.when(col('tipactividadeconomica_0') =='0115' , '01')
.when(col('tipactividadeconomica_0') =='0119' , '01')
.when(col('tipactividadeconomica_0') =='0121' , '01')
.when(col('tipactividadeconomica_0') =='0122' , '01')
.when(col('tipactividadeconomica_0') =='0123' , '01')
.when(col('tipactividadeconomica_0') =='0124' , '01')
.when(col('tipactividadeconomica_0') =='0125' , '01')
.when(col('tipactividadeconomica_0') =='0126' , '01')
.when(col('tipactividadeconomica_0') =='0127' , '01')
.when(col('tipactividadeconomica_0') =='0128' , '01')
.when(col('tipactividadeconomica_0') =='0129' , '01')
.when(col('tipactividadeconomica_0') =='0130' , '01')
.when(col('tipactividadeconomica_0') =='0141' , '01')
.when(col('tipactividadeconomica_0') =='0142' , '01')
.when(col('tipactividadeconomica_0') =='0143' , '01')
.when(col('tipactividadeconomica_0') =='0144' , '01')
.when(col('tipactividadeconomica_0') =='0145' , '01')
.when(col('tipactividadeconomica_0') =='0149' , '01')
.when(col('tipactividadeconomica_0') =='0150' , '01')
.when(col('tipactividadeconomica_0') =='0161' , '01')
.when(col('tipactividadeconomica_0') =='0162' , '01')
.when(col('tipactividadeconomica_0') =='0163' , '01')
.when(col('tipactividadeconomica_0') =='0164' , '01')
.when(col('tipactividadeconomica_0') =='0170' , '01')
.when(col('tipactividadeconomica_0') =='0210' , '02')
.when(col('tipactividadeconomica_0') =='0220' , '02')
.when(col('tipactividadeconomica_0') =='0230' , '01')
.when(col('tipactividadeconomica_0') =='0240' , '02')
.when(col('tipactividadeconomica_0') =='0311' , '05')
.when(col('tipactividadeconomica_0') =='0312' , '05')
.when(col('tipactividadeconomica_0') =='0321' , '05')
.when(col('tipactividadeconomica_0') =='0322' , '05')
.when(col('tipactividadeconomica_0') =='0510' , '10')
.when(col('tipactividadeconomica_0') =='0520' , '10')
.when(col('tipactividadeconomica_0') =='0610' , '11')
.when(col('tipactividadeconomica_0') =='0620' , '11')
.when(col('tipactividadeconomica_0') =='0710' , '13')
.when(col('tipactividadeconomica_0') =='0721' , '12')
.when(col('tipactividadeconomica_0') =='0722' , '13')
.when(col('tipactividadeconomica_0') =='0723' , '13')
.when(col('tipactividadeconomica_0') =='0729' , '13')
.when(col('tipactividadeconomica_0') =='0811' , '14')
.when(col('tipactividadeconomica_0') =='0812' , '14')
.when(col('tipactividadeconomica_0') =='0820' , '14')
.when(col('tipactividadeconomica_0') =='0891' , '14')
.when(col('tipactividadeconomica_0') =='0892' , '14')
.when(col('tipactividadeconomica_0') =='0899' , '10')
.when(col('tipactividadeconomica_0') =='0910' , '11')
.when(col('tipactividadeconomica_0') =='0990' , '10')
.when(col('tipactividadeconomica_0') =='1011' , '15')
.when(col('tipactividadeconomica_0') =='1012' , '15')
.when(col('tipactividadeconomica_0') =='1020' , '15')
.when(col('tipactividadeconomica_0') =='1030' , '15')
.when(col('tipactividadeconomica_0') =='1040' , '15')
.when(col('tipactividadeconomica_0') =='1051' , '15')
.when(col('tipactividadeconomica_0') =='1052' , '15')
.when(col('tipactividadeconomica_0') =='1061' , '15')
.when(col('tipactividadeconomica_0') =='1062' , '15')
.when(col('tipactividadeconomica_0') =='1063' , '15')
.when(col('tipactividadeconomica_0') =='1071' , '15')
.when(col('tipactividadeconomica_0') =='1072' , '15')
.when(col('tipactividadeconomica_0') =='1081' , '15')
.when(col('tipactividadeconomica_0') =='1082' , '15')
.when(col('tipactividadeconomica_0') =='1083' , '15')
.when(col('tipactividadeconomica_0') =='1084' , '15')
.when(col('tipactividadeconomica_0') =='1089' , '15')
.when(col('tipactividadeconomica_0') =='1090' , '15')
.when(col('tipactividadeconomica_0') =='1101' , '15')
.when(col('tipactividadeconomica_0') =='1102' , '15')
.when(col('tipactividadeconomica_0') =='1103' , '15')
.when(col('tipactividadeconomica_0') =='1104' , '15')
.when(col('tipactividadeconomica_0') =='1200' , '01')
.when(col('tipactividadeconomica_0') =='1311' , '17')
.when(col('tipactividadeconomica_0') =='1312' , '17')
.when(col('tipactividadeconomica_0') =='1313' , '17')
.when(col('tipactividadeconomica_0') =='1313' , '18')
.when(col('tipactividadeconomica_0') =='1391' , '17')
.when(col('tipactividadeconomica_0') =='1392' , '17')
.when(col('tipactividadeconomica_0') =='1393' , '17')
.when(col('tipactividadeconomica_0') =='1394' , '17')
.when(col('tipactividadeconomica_0') =='1399' , '17')
.when(col('tipactividadeconomica_0') =='1410' , '18')
.when(col('tipactividadeconomica_0') =='1420' , '18')
.when(col('tipactividadeconomica_0') =='1430' , '17')
.when(col('tipactividadeconomica_0') =='1511' , '18')
.when(col('tipactividadeconomica_0') =='1512' , '19')
.when(col('tipactividadeconomica_0') =='1513' , '19')
.when(col('tipactividadeconomica_0') =='1521' , '19')
.when(col('tipactividadeconomica_0') =='1522' , '19')
.when(col('tipactividadeconomica_0') =='1523' , '19')
.when(col('tipactividadeconomica_0') =='1610' , '20')
.when(col('tipactividadeconomica_0') =='1620' , '20')
.when(col('tipactividadeconomica_0') =='1630' , '20')
.when(col('tipactividadeconomica_0') =='1640' , '20')
.when(col('tipactividadeconomica_0') =='1690' , '19')
.when(col('tipactividadeconomica_0') =='1701' , '21')
.when(col('tipactividadeconomica_0') =='1702' , '21')
.when(col('tipactividadeconomica_0') =='1709' , '17')
.when(col('tipactividadeconomica_0') =='1811' , '17')
.when(col('tipactividadeconomica_0') =='1812' , '22')
.when(col('tipactividadeconomica_0') =='1820' , '22')
.when(col('tipactividadeconomica_0') =='1910' , '23')
.when(col('tipactividadeconomica_0') =='1921' , '10')
.when(col('tipactividadeconomica_0') =='2011' , '15')
.when(col('tipactividadeconomica_0') =='2012' , '24')
.when(col('tipactividadeconomica_0') =='2013' , '24')
.when(col('tipactividadeconomica_0') =='2014' , '24')
.when(col('tipactividadeconomica_0') =='2021' , '24')
.when(col('tipactividadeconomica_0') =='2022' , '24')
.when(col('tipactividadeconomica_0') =='2023' , '24')
.when(col('tipactividadeconomica_0') =='2029' , '24')
.when(col('tipactividadeconomica_0') =='2030' , '24')
.when(col('tipactividadeconomica_0') =='2100' , '23')
.when(col('tipactividadeconomica_0') =='2211' , '25')
.when(col('tipactividadeconomica_0') =='2212' , '25')
.when(col('tipactividadeconomica_0') =='2219' , '19')
.when(col('tipactividadeconomica_0') =='2221' , '25')
.when(col('tipactividadeconomica_0') =='2229' , '19')
.when(col('tipactividadeconomica_0') =='2310' , '26')
.when(col('tipactividadeconomica_0') =='2391' , '26')
.when(col('tipactividadeconomica_0') =='2392' , '26')
.when(col('tipactividadeconomica_0') =='2393' , '26')
.when(col('tipactividadeconomica_0') =='2394' , '26')
.when(col('tipactividadeconomica_0') =='2395' , '26')
.when(col('tipactividadeconomica_0') =='2396' , '26')
.when(col('tipactividadeconomica_0') =='2399' , '26')
.when(col('tipactividadeconomica_0') =='2410' , '27')
.when(col('tipactividadeconomica_0') =='2421' , '27')
.when(col('tipactividadeconomica_0') =='2429' , '23')
.when(col('tipactividadeconomica_0') =='2431' , '27')
.when(col('tipactividadeconomica_0') =='2432' , '27')
.when(col('tipactividadeconomica_0') =='2511' , '28')
.when(col('tipactividadeconomica_0') =='2512' , '28')
.when(col('tipactividadeconomica_0') =='2513' , '28')
.when(col('tipactividadeconomica_0') =='2520' , '29')
.when(col('tipactividadeconomica_0') =='2591' , '28')
.when(col('tipactividadeconomica_0') =='2592' , '28')
.when(col('tipactividadeconomica_0') =='2593' , '28')
.when(col('tipactividadeconomica_0') =='2599' , '28')
.when(col('tipactividadeconomica_0') =='2610' , '25')
.when(col('tipactividadeconomica_0') =='2620' , '30')
.when(col('tipactividadeconomica_0') =='2630' , '31')
.when(col('tipactividadeconomica_0') =='2640' , '32')
.when(col('tipactividadeconomica_0') =='2651' , '31')
.when(col('tipactividadeconomica_0') =='2652' , '33')
.when(col('tipactividadeconomica_0') =='2660' , '33')
.when(col('tipactividadeconomica_0') =='2670' , '32')
.when(col('tipactividadeconomica_0') =='2680' , '24')
.when(col('tipactividadeconomica_0') =='2711' , '31')
.when(col('tipactividadeconomica_0') =='2712' , '31')
.when(col('tipactividadeconomica_0') =='2720' , '31')
.when(col('tipactividadeconomica_0') =='2731' , '31')
.when(col('tipactividadeconomica_0') =='2732' , '25')
.when(col('tipactividadeconomica_0') =='2740' , '31')
.when(col('tipactividadeconomica_0') =='2750' , '29')
.when(col('tipactividadeconomica_0') =='2790' , '29')
.when(col('tipactividadeconomica_0') =='2811' , '29')
.when(col('tipactividadeconomica_0') =='2812' , '25')
.when(col('tipactividadeconomica_0') =='2813' , '29')
.when(col('tipactividadeconomica_0') =='2814' , '29')
.when(col('tipactividadeconomica_0') =='2815' , '29')
.when(col('tipactividadeconomica_0') =='2816' , '29')
.when(col('tipactividadeconomica_0') =='2817' , '24')
.when(col('tipactividadeconomica_0') =='2818' , '29')
.when(col('tipactividadeconomica_0') =='2819' , '29')
.when(col('tipactividadeconomica_0') =='2821' , '29')
.when(col('tipactividadeconomica_0') =='2822' , '29')
.when(col('tipactividadeconomica_0') =='2823' , '29')
.when(col('tipactividadeconomica_0') =='2824' , '29')
.when(col('tipactividadeconomica_0') =='2825' , '29')
.when(col('tipactividadeconomica_0') =='2826' , '29')
.when(col('tipactividadeconomica_0') =='2829' , '29')
.when(col('tipactividadeconomica_0') =='2910' , '34')
.when(col('tipactividadeconomica_0') =='2920' , '34')
.when(col('tipactividadeconomica_0') =='2930' , '31')
.when(col('tipactividadeconomica_0') =='3011' , '35')
.when(col('tipactividadeconomica_0') =='3012' , '35')
.when(col('tipactividadeconomica_0') =='3020' , '35')
.when(col('tipactividadeconomica_0') =='3030' , '29')
.when(col('tipactividadeconomica_0') =='3040' , '29')
.when(col('tipactividadeconomica_0') =='3091' , '35')
.when(col('tipactividadeconomica_0') =='3092' , '35')
.when(col('tipactividadeconomica_0') =='3099' , '35')
.when(col('tipactividadeconomica_0') =='3110' , '35')
.when(col('tipactividadeconomica_0') =='3120' , '36')
.when(col('tipactividadeconomica_0') =='3210' , '33')
.when(col('tipactividadeconomica_0') =='3220' , '36')
.when(col('tipactividadeconomica_0') =='3230' , '19')
.when(col('tipactividadeconomica_0') =='3240' , '35')
.when(col('tipactividadeconomica_0') =='3250' , '17')
.when(col('tipactividadeconomica_0') =='3290' , '18')
.when(col('tipactividadeconomica_0') =='3311' , '28')
.when(col('tipactividadeconomica_0') =='3312' , '29')
.when(col('tipactividadeconomica_0') =='3313' , '31')
.when(col('tipactividadeconomica_0') =='3314' , '29')
.when(col('tipactividadeconomica_0') =='3315' , '35')
.when(col('tipactividadeconomica_0') =='3319' , '17')
.when(col('tipactividadeconomica_0') =='3320' , '29')
.when(col('tipactividadeconomica_0') =='3511' , '40')
.when(col('tipactividadeconomica_0') =='3512' , '40')
.when(col('tipactividadeconomica_0') =='3513' , '40')
.when(col('tipactividadeconomica_0') =='3514' , '40')
.when(col('tipactividadeconomica_0') =='3520' , '40')
.when(col('tipactividadeconomica_0') =='3530' , '15')
.when(col('tipactividadeconomica_0') =='3600' , '41')
.when(col('tipactividadeconomica_0') =='3700' , '90')
.when(col('tipactividadeconomica_0') =='3811' , '90')
.when(col('tipactividadeconomica_0') =='3812' , '23')
.when(col('tipactividadeconomica_0') =='3821' , '24')
.when(col('tipactividadeconomica_0') =='3822' , '23')
.when(col('tipactividadeconomica_0') =='3830' , '37')
.when(col('tipactividadeconomica_0') =='3900' , '45')
.when(col('tipactividadeconomica_0') =='4111' , '45')
.when(col('tipactividadeconomica_0') =='4112' , '45')
.when(col('tipactividadeconomica_0') =='4210' , '45')
.when(col('tipactividadeconomica_0') =='4220' , '45')
.when(col('tipactividadeconomica_0') =='4290' , '45')
.when(col('tipactividadeconomica_0') =='4311' , '45')
.when(col('tipactividadeconomica_0') =='4312' , '45')
.when(col('tipactividadeconomica_0') =='4321' , '45')
.when(col('tipactividadeconomica_0') =='4322' , '45')
.when(col('tipactividadeconomica_0') =='4329' , '45')
.when(col('tipactividadeconomica_0') =='4330' , '45')
.when(col('tipactividadeconomica_0') =='4390' , '45')
.when(col('tipactividadeconomica_0') =='4511' , '50')
.when(col('tipactividadeconomica_0') =='4512' , '50')
.when(col('tipactividadeconomica_0') =='4520' , '50')
.when(col('tipactividadeconomica_0') =='4530' , '50')
.when(col('tipactividadeconomica_0') =='4541' , '50')
.when(col('tipactividadeconomica_0') =='4542' , '35')
.when(col('tipactividadeconomica_0') =='4610' , '51')
.when(col('tipactividadeconomica_0') =='4620' , '51')
.when(col('tipactividadeconomica_0') =='4631' , '51')
.when(col('tipactividadeconomica_0') =='4632' , '51')
.when(col('tipactividadeconomica_0') =='4641' , '51')
.when(col('tipactividadeconomica_0') =='4642' , '51')
.when(col('tipactividadeconomica_0') =='4643' , '51')
.when(col('tipactividadeconomica_0') =='4644' , '51')
.when(col('tipactividadeconomica_0') =='4645' , '51')
.when(col('tipactividadeconomica_0') =='4649' , '51')
.when(col('tipactividadeconomica_0') =='4651' , '51')
.when(col('tipactividadeconomica_0') =='4652' , '51')
.when(col('tipactividadeconomica_0') =='4653' , '51')
.when(col('tipactividadeconomica_0') =='4659' , '51')
.when(col('tipactividadeconomica_0') =='4661' , '51')
.when(col('tipactividadeconomica_0') =='4662' , '51')
.when(col('tipactividadeconomica_0') =='4663' , '51')
.when(col('tipactividadeconomica_0') =='4664' , '51')
.when(col('tipactividadeconomica_0') =='4665' , '51')
.when(col('tipactividadeconomica_0') =='4669' , '51')
.when(col('tipactividadeconomica_0') =='4690' , '51')
.when(col('tipactividadeconomica_0') =='4711' , '52')
.when(col('tipactividadeconomica_0') =='4719' , '52')
.when(col('tipactividadeconomica_0') =='4721' , '52')
.when(col('tipactividadeconomica_0') =='4722' , '52')
.when(col('tipactividadeconomica_0') =='4723' , '52')
.when(col('tipactividadeconomica_0') =='4724' , '52')
.when(col('tipactividadeconomica_0') =='4729' , '52')
.when(col('tipactividadeconomica_0') =='4731' , '50')
.when(col('tipactividadeconomica_0') =='4732' , '50')
.when(col('tipactividadeconomica_0') =='4741' , '52')
.when(col('tipactividadeconomica_0') =='4742' , '52')
.when(col('tipactividadeconomica_0') =='4751' , '52')
.when(col('tipactividadeconomica_0') =='4752' , '52')
.when(col('tipactividadeconomica_0') =='4753' , '52')
.when(col('tipactividadeconomica_0') =='4754' , '52')
.when(col('tipactividadeconomica_0') =='4755' , '52')
.when(col('tipactividadeconomica_0') =='4759' , '52')
.when(col('tipactividadeconomica_0') =='4761' , '52')
.when(col('tipactividadeconomica_0') =='4762' , '52')
.when(col('tipactividadeconomica_0') =='4769' , '52')
.when(col('tipactividadeconomica_0') =='4771' , '52')
.when(col('tipactividadeconomica_0') =='4772' , '52')
.when(col('tipactividadeconomica_0') =='4773' , '52')
.when(col('tipactividadeconomica_0') =='4774' , '52')
.when(col('tipactividadeconomica_0') =='4775' , '52')
.when(col('tipactividadeconomica_0') =='4781' , '52')
.when(col('tipactividadeconomica_0') =='4782' , '52')
.when(col('tipactividadeconomica_0') =='4789' , '52')
.when(col('tipactividadeconomica_0') =='4791' , '52')
.when(col('tipactividadeconomica_0') =='4792' , '52')
.when(col('tipactividadeconomica_0') =='4799' , '52')
.when(col('tipactividadeconomica_0') =='4911' , '60')
.when(col('tipactividadeconomica_0') =='4912' , '60')
.when(col('tipactividadeconomica_0') =='4921' , '60')
.when(col('tipactividadeconomica_0') =='4922' , '60')
.when(col('tipactividadeconomica_0') =='4923' , '60')
.when(col('tipactividadeconomica_0') =='4930' , '60')
.when(col('tipactividadeconomica_0') =='5011' , '61')
.when(col('tipactividadeconomica_0') =='5012' , '61')
.when(col('tipactividadeconomica_0') =='5021' , '61')
.when(col('tipactividadeconomica_0') =='5022' , '61')
.when(col('tipactividadeconomica_0') =='5111' , '62')
.when(col('tipactividadeconomica_0') =='5112' , '62')
.when(col('tipactividadeconomica_0') =='5121' , '62')
.when(col('tipactividadeconomica_0') =='5122' , '62')
.when(col('tipactividadeconomica_0') =='5210' , '63')
.when(col('tipactividadeconomica_0') =='5221' , '50')
.when(col('tipactividadeconomica_0') =='5222' , '63')
.when(col('tipactividadeconomica_0') =='5223' , '63')
.when(col('tipactividadeconomica_0') =='5224' , '63')
.when(col('tipactividadeconomica_0') =='5229' , '63')
.when(col('tipactividadeconomica_0') =='5310' , '64')
.when(col('tipactividadeconomica_0') =='5320' , '64')
.when(col('tipactividadeconomica_0') =='5511' , '55')
.when(col('tipactividadeconomica_0') =='5512' , '55')
.when(col('tipactividadeconomica_0') =='5513' , '55')
.when(col('tipactividadeconomica_0') =='5514' , '55')
.when(col('tipactividadeconomica_0') =='5519' , '55')
.when(col('tipactividadeconomica_0') =='5520' , '55')
.when(col('tipactividadeconomica_0') =='5530' , '55')
.when(col('tipactividadeconomica_0') =='5590' , '55')
.when(col('tipactividadeconomica_0') =='5611' , '55')
.when(col('tipactividadeconomica_0') =='5612' , '55')
.when(col('tipactividadeconomica_0') =='5613' , '55')
.when(col('tipactividadeconomica_0') =='5619' , '55')
.when(col('tipactividadeconomica_0') =='5621' , '55')
.when(col('tipactividadeconomica_0') =='5629' , '55')
.when(col('tipactividadeconomica_0') =='5630' , '55')
.when(col('tipactividadeconomica_0') =='5811' , '22')
.when(col('tipactividadeconomica_0') =='5812' , '22')
.when(col('tipactividadeconomica_0') =='5813' , '22')
.when(col('tipactividadeconomica_0') =='5819' , '22')
.when(col('tipactividadeconomica_0') =='5820' , '72')
.when(col('tipactividadeconomica_0') =='5911' , '92')
.when(col('tipactividadeconomica_0') =='5912' , '92')
.when(col('tipactividadeconomica_0') =='5913' , '92')
.when(col('tipactividadeconomica_0') =='5914' , '92')
.when(col('tipactividadeconomica_0') =='5920' , '22')
.when(col('tipactividadeconomica_0') =='6010' , '72')
.when(col('tipactividadeconomica_0') =='6020' , '72')
.when(col('tipactividadeconomica_0') =='6110' , '64')
.when(col('tipactividadeconomica_0') =='6120' , '64')
.when(col('tipactividadeconomica_0') =='6130' , '64')
.when(col('tipactividadeconomica_0') =='6190' , '64')
.when(col('tipactividadeconomica_0') =='6201' , '72')
.when(col('tipactividadeconomica_0') =='6202' , '72')
.when(col('tipactividadeconomica_0') =='6209' , '72')
.when(col('tipactividadeconomica_0') =='6311' , '72')
.when(col('tipactividadeconomica_0') =='6312' , '72')
.when(col('tipactividadeconomica_0') =='6391' , '92')
.when(col('tipactividadeconomica_0') =='6399' , '74')
.when(col('tipactividadeconomica_0') =='6411' , '65')
.when(col('tipactividadeconomica_0') =='6412' , '65')
.when(col('tipactividadeconomica_0') =='6421' , '65')
.when(col('tipactividadeconomica_0') =='6422' , '65')
.when(col('tipactividadeconomica_0') =='6423' , '65')
.when(col('tipactividadeconomica_0') =='6424' , '65')
.when(col('tipactividadeconomica_0') =='6431' , '67')
.when(col('tipactividadeconomica_0') =='6432' , '66')
.when(col('tipactividadeconomica_0') =='6491' , '65')
.when(col('tipactividadeconomica_0') =='6492' , '65')
.when(col('tipactividadeconomica_0') =='6493' , '65')
.when(col('tipactividadeconomica_0') =='6494' , '65')
.when(col('tipactividadeconomica_0') =='6495' , '65')
.when(col('tipactividadeconomica_0') =='6499' , '65')
.when(col('tipactividadeconomica_0') =='6511' , '66')
.when(col('tipactividadeconomica_0') =='6512' , '66')
.when(col('tipactividadeconomica_0') =='6513' , '66')
.when(col('tipactividadeconomica_0') =='6514' , '65')
.when(col('tipactividadeconomica_0') =='6521' , '66')
.when(col('tipactividadeconomica_0') =='6522' , '66')
.when(col('tipactividadeconomica_0') =='6531' , '66')
.when(col('tipactividadeconomica_0') =='6532' , '66')
.when(col('tipactividadeconomica_0') =='6611' , '67')
.when(col('tipactividadeconomica_0') =='6612' , '67')
.when(col('tipactividadeconomica_0') =='6613' , '67')
.when(col('tipactividadeconomica_0') =='6614' , '67')
.when(col('tipactividadeconomica_0') =='6615' , '67')
.when(col('tipactividadeconomica_0') =='6619' , '67')
.when(col('tipactividadeconomica_0') =='6621' , '67')
.when(col('tipactividadeconomica_0') =='6629' , '67')
.when(col('tipactividadeconomica_0') =='6630' , '67')
.when(col('tipactividadeconomica_0') =='6810' , '70')
.when(col('tipactividadeconomica_0') =='6820' , '70')
.when(col('tipactividadeconomica_0') =='6910' , '74')
.when(col('tipactividadeconomica_0') =='6920' , '74')
.when(col('tipactividadeconomica_0') =='7010' , '74')
.when(col('tipactividadeconomica_0') =='7020' , '74')
.when(col('tipactividadeconomica_0') =='7110' , '74')
.when(col('tipactividadeconomica_0') =='7120' , '74')
.when(col('tipactividadeconomica_0') =='7210' , '73')
.when(col('tipactividadeconomica_0') =='7220' , '73')
.when(col('tipactividadeconomica_0') =='7310' , '74')
.when(col('tipactividadeconomica_0') =='7320' , '74')
.when(col('tipactividadeconomica_0') =='7410' , '74')
.when(col('tipactividadeconomica_0') =='7420' , '74')
.when(col('tipactividadeconomica_0') =='7490' , '74')
.when(col('tipactividadeconomica_0') =='7500' , '85')
.when(col('tipactividadeconomica_0') =='7710' , '71')
.when(col('tipactividadeconomica_0') =='7721' , '71')
.when(col('tipactividadeconomica_0') =='7722' , '71')
.when(col('tipactividadeconomica_0') =='7729' , '71')
.when(col('tipactividadeconomica_0') =='7730' , '71')
.when(col('tipactividadeconomica_0') =='7740' , '65')
.when(col('tipactividadeconomica_0') =='7810' , '74')
.when(col('tipactividadeconomica_0') =='7820' , '74')
.when(col('tipactividadeconomica_0') =='7830' , '74')
.when(col('tipactividadeconomica_0') =='7911' , '63')
.when(col('tipactividadeconomica_0') =='7912' , '63')
.when(col('tipactividadeconomica_0') =='7990' , '63')
.when(col('tipactividadeconomica_0') =='8010' , '74')
.when(col('tipactividadeconomica_0') =='8020' , '74')
.when(col('tipactividadeconomica_0') =='8030' , '74')
.when(col('tipactividadeconomica_0') =='8110' , '70')
.when(col('tipactividadeconomica_0') =='8121' , '74')
.when(col('tipactividadeconomica_0') =='8129' , '74')
.when(col('tipactividadeconomica_0') =='8130' , '01')
.when(col('tipactividadeconomica_0') =='8211' , '74')
.when(col('tipactividadeconomica_0') =='8219' , '64')
.when(col('tipactividadeconomica_0') =='8220' , '74')
.when(col('tipactividadeconomica_0') =='8230' , '74')
.when(col('tipactividadeconomica_0') =='8291' , '74')
.when(col('tipactividadeconomica_0') =='8292' , '74')
.when(col('tipactividadeconomica_0') =='8299' , '74')
.when(col('tipactividadeconomica_0') =='8411' , '75')
.when(col('tipactividadeconomica_0') =='8412' , '75')
.when(col('tipactividadeconomica_0') =='8413' , '75')
.when(col('tipactividadeconomica_0') =='8414' , '75')
.when(col('tipactividadeconomica_0') =='8415' , '75')
.when(col('tipactividadeconomica_0') =='8421' , '75')
.when(col('tipactividadeconomica_0') =='8422' , '75')
.when(col('tipactividadeconomica_0') =='8423' , '75')
.when(col('tipactividadeconomica_0') =='8424' , '75')
.when(col('tipactividadeconomica_0') =='8430' , '75')
.when(col('tipactividadeconomica_0') =='8512' , '80')
.when(col('tipactividadeconomica_0') =='8513' , '80')
.when(col('tipactividadeconomica_0') =='8521' , '80')
.when(col('tipactividadeconomica_0') =='8522' , '80')
.when(col('tipactividadeconomica_0') =='8523' , '80')
.when(col('tipactividadeconomica_0') =='8530' , '80')
.when(col('tipactividadeconomica_0') =='8541' , '80')
.when(col('tipactividadeconomica_0') =='8542' , '80')
.when(col('tipactividadeconomica_0') =='8543' , '80')
.when(col('tipactividadeconomica_0') =='8544' , '80')
.when(col('tipactividadeconomica_0') =='8551' , '80')
.when(col('tipactividadeconomica_0') =='8552' , '80')
.when(col('tipactividadeconomica_0') =='8553' , '80')
.when(col('tipactividadeconomica_0') =='8559' , '80')
.when(col('tipactividadeconomica_0') =='8560' , '74')
.when(col('tipactividadeconomica_0') =='8610' , '85')
.when(col('tipactividadeconomica_0') =='8621' , '85')
.when(col('tipactividadeconomica_0') =='8622' , '85')
.when(col('tipactividadeconomica_0') =='8691' , '85')
.when(col('tipactividadeconomica_0') =='8692' , '85')
.when(col('tipactividadeconomica_0') =='8699' , '85')
.when(col('tipactividadeconomica_0') =='8710' , '85')
.when(col('tipactividadeconomica_0') =='8720' , '85')
.when(col('tipactividadeconomica_0') =='8730' , '85')
.when(col('tipactividadeconomica_0') =='8790' , '85')
.when(col('tipactividadeconomica_0') =='8810' , '85')
.when(col('tipactividadeconomica_0') =='8890' , '85')
.when(col('tipactividadeconomica_0') =='9001' , '92')
.when(col('tipactividadeconomica_0') =='9002' , '92')
.when(col('tipactividadeconomica_0') =='9003' , '92')
.when(col('tipactividadeconomica_0') =='9004' , '92')
.when(col('tipactividadeconomica_0') =='9005' , '92')
.when(col('tipactividadeconomica_0') =='9006' , '92')
.when(col('tipactividadeconomica_0') =='9007' , '92')
.when(col('tipactividadeconomica_0') =='9008' , '92')
.when(col('tipactividadeconomica_0') =='9101' , '92')
.when(col('tipactividadeconomica_0') =='9102' , '92')
.when(col('tipactividadeconomica_0') =='9103' , '92')
.when(col('tipactividadeconomica_0') =='9200' , '92')
.when(col('tipactividadeconomica_0') =='9311' , '92')
.when(col('tipactividadeconomica_0') =='9312' , '92')
.when(col('tipactividadeconomica_0') =='9319' , '92')
.when(col('tipactividadeconomica_0') =='9321' , '92')
.when(col('tipactividadeconomica_0') =='9329' , '92')
.when(col('tipactividadeconomica_0') =='9411' , '91')
.when(col('tipactividadeconomica_0') =='9412' , '91')
.when(col('tipactividadeconomica_0') =='9420' , '91')
.when(col('tipactividadeconomica_0') =='9491' , '91')
.when(col('tipactividadeconomica_0') =='9492' , '91')
.when(col('tipactividadeconomica_0') =='9499' , '01')
.when(col('tipactividadeconomica_0') =='9511' , '72')
.when(col('tipactividadeconomica_0') =='9512' , '32')
.when(col('tipactividadeconomica_0') =='9521' , '52')
.when(col('tipactividadeconomica_0') =='9522' , '52')
.when(col('tipactividadeconomica_0') =='9523' , '52')
.when(col('tipactividadeconomica_0') =='9524' , '36')
.when(col('tipactividadeconomica_0') =='9529' , '36')
.when(col('tipactividadeconomica_0') =='9601' , '93')
.when(col('tipactividadeconomica_0') =='9602' , '93')
.when(col('tipactividadeconomica_0') =='9603' , '93')
.when(col('tipactividadeconomica_0') =='9609' , '93')
.when(col('tipactividadeconomica_0') =='9700' , '95')
.when(col('tipactividadeconomica_0') =='9810' , '96')
.when(col('tipactividadeconomica_0') =='9820' , '97')
.when(col('tipactividadeconomica_0') =='9900' , '99')
.when(col('tipactividadeconomica_0') =='0116' , '01')
.when(col('tipactividadeconomica_0') =='0146' , '01')
.when(col('tipactividadeconomica_0') =='0810' , '14')
.when(col('tipactividadeconomica_0') =='0893' , '14')
.when(col('tipactividadeconomica_0') =='1010' , '15')
.when(col('tipactividadeconomica_0') =='1050' , '15')
.when(col('tipactividadeconomica_0') =='1073' , '15')
.when(col('tipactividadeconomica_0') =='1074' , '15')
.when(col('tipactividadeconomica_0') =='1075' , '15')
.when(col('tipactividadeconomica_0') =='1079' , '15')
.when(col('tipactividadeconomica_0') =='1080' , '15')
.when(col('tipactividadeconomica_0') =='1520' , '19')
.when(col('tipactividadeconomica_0') =='1621' , '20')
.when(col('tipactividadeconomica_0') =='1622' , '20')
.when(col('tipactividadeconomica_0') =='1623' , '20')
.when(col('tipactividadeconomica_0') =='1629' , '20')
.when(col('tipactividadeconomica_0') =='1920' , '23')
.when(col('tipactividadeconomica_0') =='2220' , '25')
.when(col('tipactividadeconomica_0') =='2420' , '27')
.when(col('tipactividadeconomica_0') =='2710' , '31')
.when(col('tipactividadeconomica_0') =='2733' , '31')
.when(col('tipactividadeconomica_0') =='3100' , '36')
.when(col('tipactividadeconomica_0') =='3211' , '36')
.when(col('tipactividadeconomica_0') =='3212' , '36')
.when(col('tipactividadeconomica_0') =='3510' , '40')
.when(col('tipactividadeconomica_0') =='4100' , '45')
.when(col('tipactividadeconomica_0') =='4510' , '50')
.when(col('tipactividadeconomica_0') =='4540' , '50')
.when(col('tipactividadeconomica_0') =='4630' , '51')
.when(col('tipactividadeconomica_0') =='4730' , '50')
.when(col('tipactividadeconomica_0') =='4763' , '52')
.when(col('tipactividadeconomica_0') =='4764' , '52')
.when(col('tipactividadeconomica_0') =='5110' , '62')
.when(col('tipactividadeconomica_0') =='5120' , '62')
.when(col('tipactividadeconomica_0') =='5510' , '55')
.when(col('tipactividadeconomica_0') =='5610' , '55')
.when(col('tipactividadeconomica_0') =='6419' , '65')
.when(col('tipactividadeconomica_0') =='6420' , '65')
.when(col('tipactividadeconomica_0') =='6430' , '65')
.when(col('tipactividadeconomica_0') =='6520' , '66')
.when(col('tipactividadeconomica_0') =='6530' , '66')
.when(col('tipactividadeconomica_0') =='6622' , '67')
.when(col('tipactividadeconomica_0') =='8510' , '80')
.when(col('tipactividadeconomica_0') =='8549' , '80')
.when(col('tipactividadeconomica_0') =='8550' , '74')
.when(col('tipactividadeconomica_0') =='8620' , '85')
.when(col('tipactividadeconomica_0') =='8690' , '85')
)

hip_pyme_2 = hip_pyme_2.withColumn("ANT_CIIU_DEF", 
    when(col("ANT_GRUPO_CIIU").isin("01", "02"), 1)
    .when(col("ANT_GRUPO_CIIU") == "05", 2)
    .when(col("ANT_GRUPO_CIIU").isin("10", "11", "12", "13", "14"), 3)
    .when(col("ANT_GRUPO_CIIU").isin(
        "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28",
        "29", "30", "31", "32", "33", "34", "35", "36", "37"), 4)
    .when(col("ANT_GRUPO_CIIU").isin("40", "41"), 5)
    .when(col("ANT_GRUPO_CIIU") == "45", 6)
    .when(col("ANT_GRUPO_CIIU").isin("50", "51", "52"), 7)
    .when(col("ANT_GRUPO_CIIU") == "55", 8)
    .when(col("ANT_GRUPO_CIIU").isin("60", "61", "62", "63", "64"), 9)
    .when(col("ANT_GRUPO_CIIU").isin("65", "66", "67"), 10)
    .when(col("ANT_GRUPO_CIIU") == "70", 11)
    .when(col("ANT_GRUPO_CIIU").isin("71", "72", "73", "75", "80", "85", "90", "91", "92", "93", "95", "99"), 12)
    .when(col("ANT_GRUPO_CIIU") == "74", 13)
)

hip_pyme_2 = hip_pyme_2.withColumn("ANT_ID_CIIU",
    when(col("ANT_CIIU_DEF") == 1, "AGROPECUARIO")
    .when(col("ANT_CIIU_DEF") == 2, "PESCA")
    .when(col("ANT_CIIU_DEF") == 3, "MINERIA E HIDROCARBUROS")
    .when(col("ANT_CIIU_DEF") == 4, "MANUFACTURA")
    .when(col("ANT_CIIU_DEF") == 5, "SUMINISTRO DE SERV PUBLICOS")
    .when(col("ANT_CIIU_DEF") == 6, "CONSTRUCCION")
    .when(col("ANT_CIIU_DEF") == 7, "COMERCIO")
    .when(col("ANT_CIIU_DEF") == 8, "HOTELES Y RESTAURANTES")
    .when(col("ANT_CIIU_DEF") == 9, "TRANSPORTE Y COMUNICACIONES")
    .when(col("ANT_CIIU_DEF") == 10, "INTERMEDIACION FINANCIERA")
    .when(col("ANT_CIIU_DEF") == 11, "INMOBILIARIAS")
    .when(col("ANT_CIIU_DEF") == 12, "OTROS SERVICIOS")
    .when(col("ANT_CIIU_DEF") == 13, "OTRAS ACTIVIDADES EMPRESARIALES")
)

In [0]:
hip_pyme_2.createOrReplaceTempView("hip_app_pyme")

spark.sql('''
    CREATE OR REPLACE TABLE
        catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_hm_app_hippyme
    USING DELTA LOCATION 
        'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/T39290/Seguimiento/02_Applicant_Hipotecario/hm_app_hippyme'
    AS SELECT * FROM hip_app_pyme
    '''
)